# Projeto PAS - Notebook 01: Parser de PDF e Limpeza de Dados

**Objetivo:** Este *notebook* executa o *pipeline* de extração, transformação e carga (ETL). Sua responsabilidade é processar múltiplos arquivos PDF de resultados do Cebraspe (um para cada triênio) e salvá-los como um único `DataFrame` mestre, limpo e padronizado.

**Input:** Múltiplos arquivos `.pdf` (ex: `PAS_2022_2024.pdf`)

**Output:** Um único `DataFrame` mestre: `PAS_MESTRE_LIMPO_FINAL.csv`

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.5 MB/s eta 0:00:00


### Tarefa 1: Setup - Conectando ao Google Drive
Como os arquivos PDF de origem estão armazenados no Google Drive, o primeiro passo é montar (conectar) o *notebook* ao sistema de arquivos do Drive.

In [ ]:
import pdfplumber
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Tarefa 2: A Ferramenta (A Função "Parser")

Esta é a função principal de limpeza, `parse_pas_pdf`. Ela foi projetada para ser robusta e lidar com as inconsistências de formato encontradas ao longo dos 7 anos de relatórios em PDF.

Ela implementa 4 lógicas de engenharia de dados para tratar problemas complexos identificados durante a prototipagem:

1.  **Tratamento de "Linha Quebrada":** Os dados de um único aluno podem ser quebrados em múltiplas linhas no PDF.
    * **Solução:** O *parser* primeiro junta (`.join()`) todo o texto da página em uma "MegaString" para garantir a integridade dos dados antes do processamento.
2.  **Identificação do "Separador":** O delimitador de aluno real não é uma linha nova (`\n`), mas sim o caractere ` / `.
    * **Solução:** A "MegaString" é dividida (`.split(' / ')`) usando este delimitador.
3.  **Detecção de "Âncora" (Robusta):** Os cabeçalhos dos PDFs têm tamanhos inconsistentes (de 31 a 7097 linhas).
    * **Solução:** O *parser* "caça" por âncoras de texto (como "ADMINISTRAÇÃO (BACHARELADO)" ou "1.1.1.1") para encontrar dinamicamente o início dos dados reais.
4.  **Tratamento de Dados "Irregulares":** Alunos têm um número variável de colunas.
    * **Solução:** O *parser* implementa "acolchoamento" (padding), detectando o comprimento máximo (`max_cols`) e preenchendo linhas mais curtas com `None` para criar um `DataFrame` retangular.

A função também aplica a limpeza final (conversão de tipo para `float` com `pd.to_numeric` e remoção de alunos eliminados com `.dropna()`).

In [ ]:
import pdfplumber
import pandas as pd
import re
import numpy as np

def parse_pas_pdf(caminho_pdf, ano_trienio):
    print(f"\n--- Processando Triênio: {ano_trienio} ---")
    print(f"Lendo arquivo: {caminho_pdf}")

    todas_as_linhas = []
    try:
        with pdfplumber.open(caminho_pdf) as pdf:
            for pagina in pdf.pages:
                texto_da_pagina = pagina.extract_text()
                if texto_da_pagina:
                    linhas_da_pagina = texto_da_pagina.split('\n')
                    todas_as_linhas.extend(linhas_da_pagina)
    except Exception as e:
        print(f"!!! ERRO AO LER O PDF: {e}")
        return None

    indice_de_inicio = -1

    for i, linha in enumerate(todas_as_linhas):
        linha_upper = linha.upper()

        if "1.1.1.1" in linha and "ADMINISTRAÇÃO" in linha_upper:
            print(f"Encontrada 'âncora' (Padrão 2018) na linha {i}.")
            indice_de_inicio = i
            break

        elif "ADMINISTRAÇÃO (BACHARELADO)" == linha_upper:
            print(f"Encontrada 'âncora' (Padrão 2024) na linha {i}.")
            indice_de_inicio = i
            break

    if indice_de_inicio == -1:
        print(f"!!! ERRO: Não foi possível achar a 'âncora' (ADMINISTRAÇÃO ou 1.1.1.1) no PDF de {ano_trienio}.")
        return None

    linhas_de_dados = todas_as_linhas[indice_de_inicio:]
    megastring = " ".join(linhas_de_dados)
    lista_de_alunos_sujos = megastring.split(' / ')

    padrao_aluno = re.compile(r"^(\d{8}), ([\w\s'-]+), ([\d\., -]+.*)")
    padrao_split_notas = re.compile(r", ?")

    dados_limpos_listas = []
    for aluno_str in lista_de_alunos_sujos:
        aluno_str_limpo = aluno_str.strip()
        match = re.search(padrao_aluno, aluno_str_limpo)

        if match:
            inscricao = match.group(1); nome = match.group(2).strip()
            notas_str = match.group(3); notas_lista = re.split(padrao_split_notas, notas_str)
            linha_final = [inscricao, nome] + notas_lista
            dados_limpos_listas.append(linha_final)

    if not dados_limpos_listas:
        print(f"Nenhum aluno encontrado com o padrão Regex. O formato do PDF de {ano_trienio} pode ser diferente.")
        return None

    max_cols = 0
    for linha in dados_limpos_listas: max_cols = max(max_cols, len(linha))

    if max_cols < 12:
        print(f"!!! ERRO: O parser encontrou dados, mas as colunas parecem erradas (max_cols = {max_cols}).")
        return None

    dados_preenchidos = []
    for linha in dados_limpos_listas:
        colunas_faltando = max_cols - len(linha)
        linha_preenchida = linha + ([None] * colunas_faltando)
        dados_preenchidos.append(linha_preenchida)

    colunas_nomes = ['Inscricao', 'Nome', 'P1_PAS1', 'P2_PAS1', 'Red_PAS1', 'P1_PAS2', 'P2_PAS2', 'Red_PAS2', 'P1_PAS3', 'P2_PAS3', 'Red_PAS3', 'Arg_Final']
    colunas_lixo = [f'Class_{i+1}' for i in range(max_cols - len(colunas_nomes))]
    colunas_final = colunas_nomes + colunas_lixo

    df = pd.DataFrame(dados_preenchidos, columns=colunas_final)

    df_fatiado = df[colunas_nomes].copy()

    colunas_para_converter = [
        'P1_PAS1', 'P2_PAS1', 'Red_PAS1', 'P1_PAS2', 'P2_PAS2', 'Red_PAS2',
        'P1_PAS3', 'P2_PAS3', 'Red_PAS3', 'Arg_Final'
    ]
    for coluna in colunas_para_converter:
        df_fatiado.loc[:, coluna] = pd.to_numeric(df_fatiado[coluna], errors='coerce')

    df_limpo = df_fatiado.dropna(subset=colunas_para_converter).copy()

    df_limpo.loc[:, 'Ano_Trienio'] = ano_trienio

    print(f"Limpeza concluída. Encontramos {len(df_limpo)} alunos 'sobreviventes'.")

    return df_limpo


### Tarefa 3: Processamento em Lote

Esta célula executa o *pipeline* de limpeza.

Ela utiliza a função `parse_pas_pdf` (definida acima) e a aplica iterativamente a um dicionário (`arquivos_para_processar`) que mapeia cada triênio ao seu respectivo nome de arquivo PDF.

Cada *dataset* de ano limpo é salvo individualmente como um `.csv` (ex: `PAS_2022-2024_LIMPO.csv`).

In [ ]:
lista_de_dataframes = []

base_path = '/content/drive/MyDrive/Projeto PAS/'

arquivos_para_processar = {
    '2022-2024': 'Ed_38_2024_PAS_3_2022-2024_Res_final_não_eliminados.pdf',
    '2021-2023': 'Ed_27_PAS_3_2021_2023_Res_final_tipo_D_redação.pdf',
    '2020-2022': 'Ed_30_PAS_3_2020_2022_Res_Final_Tipo D_Redação.pdf',
    '2019-2021': 'Ed_30_PAS_3_2019_2021_Res_Final_Tipo D_Redação.pdf',
    '2018-2020': 'ED_37_PAS_3 _2018 -2020_Final_Tipo_D_Redacao.pdf',
    '2017-2019': 'Ed_36_PAS_3 _2017 -2019_Res_final_tipo_D_redacao_rel_nao_elimin.pdf',
    '2016-2018': 'Ed_31_2016-2018_PAS_3_Res_final_nao_eliminados.pdf'
}


print(f"--- Iniciando Fase 1.B: Processamento em Lote ---")

for ano, nome_arquivo in arquivos_para_processar.items():

    caminho_completo = base_path + nome_arquivo

    df_limpo_ano = parse_pas_pdf(caminho_pdf=caminho_completo, ano_trienio=ano)

    if df_limpo_ano is not None:

        caminho_csv = f"{base_path}PAS_{ano}_LIMPO.csv"
        df_limpo_ano.to_csv(caminho_csv, index=False)
        print(f"SUCESSO! Arquivo '{caminho_csv}' salvo.")

        lista_de_dataframes.append(df_limpo_ano)

if lista_de_dataframes:
    print("\n--- Juntando todos os triênios em um DataFrame Mestre ---")

    df_mestre = pd.concat(lista_de_dataframes, ignore_index=True)

    caminho_mestre_csv = f"{base_path}PAS_MESTRE_LIMPO.csv"
    df_mestre.to_csv(caminho_mestre_csv, index=False)

    print(f"\n--- PROJETO (PARSER) CONCLUÍDO! ---")
    print(f"Arquivo Mestre '{caminho_mestre_csv}' salvo.")
    print(f"Total de alunos 'sobreviventes' em todos os anos: {len(df_mestre)}")
    print("\nAmostra do DataFrame Mestre:")
    print(df_mestre.head())
    print(df_mestre.tail())

else:
    print("Nenhum arquivo foi processado com sucesso.")

--- Iniciando Fase 1.B: Processamento em Lote ---

--- Processando Triênio: 2022-2024 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/Ed_38_2024_PAS_3_2022-2024_Res_final_não_eliminados.pdf
Encontrada 'âncora' de início dos dados na linha 31.


/tmp/ipython-input-1319857419.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo['Ano_Trienio'] = ano_trienio


Limpeza concluída. Encontramos 8119 alunos 'sobreviventes'.
SUCESSO! Arquivo '/content/drive/MyDrive/Projeto PAS/PAS_2022-2024_LIMPO.csv' salvo.

--- Processando Triênio: 2021-2023 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/Ed_27_PAS_3_2021_2023_Res_final_tipo_D_redação.pdf
Encontrada 'âncora' de início dos dados na linha 4875.


/tmp/ipython-input-1319857419.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo['Ano_Trienio'] = ano_trienio


Limpeza concluída. Encontramos 7630 alunos 'sobreviventes'.
SUCESSO! Arquivo '/content/drive/MyDrive/Projeto PAS/PAS_2021-2023_LIMPO.csv' salvo.

--- Processando Triênio: 2020-2022 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/Ed_30_PAS_3_2020_2022_Res_Final_Tipo D_Redação.pdf
Encontrada 'âncora' de início dos dados na linha 4920.


...

Limpeza concluída. Encontramos 8105 alunos 'sobreviventes'.
SUCESSO! Arquivo '/content/drive/MyDrive/Projeto PAS/PAS_2019-2021_LIMPO.csv' salvo.

--- Processando Triênio: 2018-2020 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/ED_37_PAS_3 _2018 -2020_Final_Tipo_D_Redacao.pdf
Encontrada 'âncora' de início dos dados na linha 3772.
Limpeza concluída. Encontramos 5556 alunos 'sobreviventes'.


/tmp/ipython-input-1319857419.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo['Ano_Trienio'] = ano_trienio


SUCESSO! Arquivo '/content/drive/MyDrive/Projeto PAS/PAS_2018-2020_LIMPO.csv' salvo.

--- Processando Triênio: 2017-2019 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/Ed_36_PAS_3 _2017 -2019_Res_final_tipo_D_redacao_rel_nao_elimin.pdf
Encontrada 'âncora' de início dos dados na linha 7097.


/tmp/ipython-input-1319857419.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo['Ano_Trienio'] = ano_trienio


Limpeza concluída. Encontramos 9300 alunos 'sobreviventes'.
SUCESSO! Arquivo '/content/drive/MyDrive/Projeto PAS/PAS_2017-2019_LIMPO.csv' salvo.

--- Processando Triênio: 2016-2018 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/Ed_31_2016-2018_PAS_3_Res_final_nao_eliminados.pdf
!!! ERRO: Não foi possívelADMINISTRAÇÃO (BACHARELADO) achar a 'âncora' (ex: ADMINISTRAÇÃO) no PDF de 2016-2018.
    O parser pode precisar de uma nova 'âncora' para este ano.

--- Juntando todos os triênios em um DataFrame Mestre ---

--- PROJETO (PARSER) CONCLUÍDO! ---
Arquivo Mestre '/content/drive/MyDrive/Projeto PAS/PAS_MESTRE_LIMPO.csv' salvo.
Total de alunos 'sobreviventes' em todos os anos: 45564

Amostra do DataFrame Mestre:
  Inscricao                                  Nome P1_PAS1 P2_PAS1 Red_PAS1  \
0  22103536              Admilson Vieira de Moura   4.385  13.447    5.567   
1  22121163          Alex Vitor Goncalves Barbosa   1.169   9.061    5.799   
2  22125011              Alexandre Lobo Par

### Tarefa 4: Resgate de Dados (Debug de 2016-2018)

O *parser* V2.3 falhou no PDF de 2018, pois a "âncora" de texto era diferente (`1.1.1.1 ADMINISTRAÇÃO...`). Esta célula foi usada para "resgatar" cirurgicamente apenas aquele arquivo, aplicando a lógica de *parser* V2.4 (definida na Célula 2), sem a necessidade de re-processar os 6 arquivos que já haviam sido concluídos.

In [ ]:
base_path = '/content/drive/MyDrive/Projeto PAS/'

ano_falho = '2016-2018'
arquivo_falho = 'Ed_31_2016-2018_PAS_3_Res_final_nao_eliminados.pdf'

caminho_completo = base_path + arquivo_falho

df_resgatado = parse_pas_pdf(caminho_pdf=caminho_completo, ano_trienio=ano_falho)

if df_resgatado is not None:
    caminho_csv = f"{base_path}PAS_{ano_falho}_LIMPO.csv"
    df_resgatado.to_csv(caminho_csv, index=False)
    print(f"SUCESSO! Arquivo '{caminho_csv}' salvo.")
else:
    print(f"--- FALHA NO RESGATE. O parser V2.3 ainda não é bom o suficiente. ---")



--- Processando Triênio: 2016-2018 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/Ed_31_2016-2018_PAS_3_Res_final_nao_eliminados.pdf
Encontrada 'âncora' (Padrão 2018) na linha 35.
Limpeza concluída. Encontramos 3194 alunos 'sobreviventes'.
SUCESSO! Arquivo '/content/drive/MyDrive/Projeto PAS/PAS_2016-2018_LIMPO.csv' salvo.


### Tarefa 5: Consolidação Final (O "DataFrame Mestre")

Este é o passo final do *pipeline* de engenharia.

**Desafio de Consolidação:** O método de consolidação precisa ser "idempotente" (seguro para rodar múltiplas vezes). Uma busca genérica (`glob`) pelos arquivos `.csv` criados pode acidentalmente incluir os próprios arquivos "Mestre" de saídas anteriores, criando duplicatas.

**Solução:**
1.  **Limpeza de Artefatos:** Arquivos `_MESTRE_` de saídas anteriores são removidos do diretório.
2.  **"Caçador Inteligente" (Glob):** O padrão `glob` é refinado para `PAS_????-????_LIMPO.csv`. Este padrão é específico e "caça" *apenas* os 7 arquivos de triênio, ignorando qualquer outro arquivo.
3.  **Resultado:** `pd.concat()` é usado para "empilhar" os 7 CSVs limpos em um `DataFrame` mestre final (`PAS_MESTRE_LIMPO_FINAL.csv`), contendo **48.758 alunos**.

In [ ]:
base_path = '/content/drive/MyDrive/Projeto PAS/'

ano_regenerar = '2022-2024'
arquivo_regenerar = 'Ed_38_2024_PAS_3_2022-2024_Res_final_nao_eliminados.pdf'

caminho_completo = base_path + arquivo_regenerar

df_regenerado = parse_pas_pdf(caminho_pdf=caminho_completo, ano_trienio=ano_regenerar)

if df_regenerado is not None:
    caminho_csv = f"{base_path}PAS_{ano_regenerar}_LIMPO.csv"
    df_regenerado.to_csv(caminho_csv, index=False)
    print(f"SUCESSO! Arquivo '{caminho_csv}' (V2.4) salvo.")
else:
    print(f"--- FALHA NA REGENERAÇÃO. O parser V2.4 falhou no 2024. ---")



--- Processando Triênio: 2022-2024 ---
Lendo arquivo: /content/drive/MyDrive/Projeto PAS/Ed_38_2024_PAS_3_2022-2024_Res_final_não_eliminados.pdf
Encontrada 'âncora' (Padrão 2024) na linha 31.
Limpeza concluída. Encontramos 8119 alunos 'sobreviventes'.
SUCESSO! Arquivo '/content/drive/MyDrive/Projeto PAS/PAS_2022-2024_LIMPO.csv' (V2.4) salvo.


In [ ]:
import pandas as pd
import numpy as np
import glob

base_path = '/content/drive/MyDrive/Projeto PAS/'

padrao_de_busca = base_path + 'PAS_????-????_LIMPO.csv'

arquivos_csv_limpos = glob.glob(padrao_de_busca)

if not arquivos_csv_limpos:
    print("ERRO: Nenhum arquivo .csv limpo foi encontrado.")
else:
    print(f"Encontrados {len(arquivos_csv_limpos)} arquivos CSV de triênios para juntar:")
    print(sorted(arquivos_csv_limpos))

    lista_de_dataframes = []

    for arquivo_csv in arquivos_csv_limpos:
        df_ano = pd.read_csv(arquivo_csv)
        lista_de_dataframes.append(df_ano)

    df_mestre = pd.concat(lista_de_dataframes, ignore_index=True)

    caminho_mestre_csv = f"{base_path}PAS_MESTRE_LIMPO_FINAL.csv"
    df_mestre.to_csv(caminho_mestre_csv, index=False)

    print("\n--- PROJETO (PARSER) CONCLUÍDO! ---")
    print(f"Arquivo Mestre '{caminho_mestre_csv}' salvo.")

    print("\n--- Informações do DataFrame Mestre ---")
    df_mestre.info()

    print("\n--- Composição do DataFrame Mestre (Contagem por Ano) ---")
    print(df_mestre['Ano_Trienio'].value_counts())


--- Iniciando Tarefa 19.B: Consolidação Final (Pós-Limpeza) ---
Encontrados 7 arquivos CSV de triênios para juntar:
['/content/drive/MyDrive/Projeto PAS/PAS_2016-2018_LIMPO.csv', '/content/drive/MyDrive/Projeto PAS/PAS_2017-2019_LIMPO.csv', '/content/drive/MyDrive/Projeto PAS/PAS_2018-2020_LIMPO.csv', '/content/drive/MyDrive/Projeto PAS/PAS_2019-2021_LIMPO.csv', '/content/drive/MyDrive/Projeto PAS/PAS_2020-2022_LIMPO.csv', '/content/drive/MyDrive/Projeto PAS/PAS_2021-2023_LIMPO.csv', '/content/drive/MyDrive/Projeto PAS/PAS_2022-2024_LIMPO.csv']

--- PROJETO (PARSER) CONCLUÍDO! ---
Arquivo Mestre '/content/drive/MyDrive/Projeto PAS/PAS_MESTRE_LIMPO_FINAL.csv' salvo.

--- Informações do DataFrame Mestre ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48758 entries, 0 to 48757
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Inscricao    48758 non-null  int64  
 1   Nome         48758 non-null  object 
 2   P1_